In [1]:
import pickle
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../..')
import os
import subprocess
from importlib import reload
import pickle

In [2]:
colnames=['article_id', 'title', 'abstract']
        
df_abst = pd.read_csv("/home/hilal/Documents/chem_relation_extractor/data/chemprot_development/chemprot_development_abstracts.tsv", sep="\t", names=colnames, header=None)
df_abst.head()

,article_id,title,abstract
0,10064839,Binding of dimemorfan to sigma-1 receptor and ...,Dextromethorphan ((+)-3-methoxy-N-methylmorphi...
1,10082498,Angiotensin II receptor blockade in normotensi...,Use of angiotensin (Ang) II AT1 receptor antag...
2,10193663,Characterisation of the 5-HT receptor binding ...,The affinity of eletriptan ((R)-3-(1-methyl-2-...
3,10193665,Pharmacological profile of neuroleptics at hum...,"Using radioligand binding techniques, we deter..."
4,10226872,Disodium cromoglycate does not prevent terbuta...,"In humans, prolonged administration of the bet..."


In [11]:
for i in df_abst.index:
    file = open( "input_dev/" + str(df_abst.loc[i,'article_id']) + ".txt", "w")
    article_id = df_abst.loc[i,'article_id']
    inp = list(df_abst[df_abst['article_id'] == article_id]['title'] + ' ' + df_abst[df_abst['article_id'] == article_id]['abstract'])[0]
    file.write(inp)


In [12]:
for i in df_abst.index:
    cmd = "./geniass "+ "input_dev/" + str(df_abst.loc[i,'article_id']) + ".txt " + "output_dev/" + str(df_abst.loc[i,'article_id']) + "_o.txt"
    output = subprocess.check_output(cmd, shell = True)
    lines = output.decode().split("\n")
    output = [i.split('\t')[0] for i in lines if i.split('\t')[0]]
    file.close()


In [13]:
df_sentence = df_abst[['article_id']].copy()
df_sentence['sent_id'] = 0
df_sentence['sentence'] = ''

for i in df_abst.index:
    file = open("output_dev/"+str(df_abst.loc[i,'article_id']) + "_o.txt", "r")
    sent_list = file.readlines()
    df_temp = pd.DataFrame({'article_id':df_abst.loc[i,'article_id'], 'sent_id':np.arange(len(sent_list)) })
    for j in range(len(sent_list)):
        df_temp.loc[df_temp['sent_id'] == j, 'sentence'] = sent_list[j].replace('\n', '')
    df_sentence = df_sentence.append([df_temp])
    
df_sentence.drop_duplicates(subset = ['article_id', 'sent_id'], keep = 'last', inplace = True)
   

In [14]:
with open('geniass_sentence_split_development.pickle', 'wb' ) as f:
    pickle.dump(df_sentence, f, protocol = pickle.HIGHEST_PROTOCOL)